In [38]:
from laboneq.simple import *
from helpers.example_notebook_helper import *
from helpers.example_notebook_simple import create_device_setup
#from zhinst.toolkit import Session
from config import config
name = "q0"

In [79]:
def make_device_setup():
    descriptor=f"""\
    instrument_list:
        SHFQA:
        - address: DEV12096
          uid: device_shfqa
        SHFSG:
        - address: DEV12173
          uid: device_shfsg
        PQSC:  
        - address: DEV10084
          uid: device_pqsc

    connections:
        device_shfqa:    
            - iq_signal: q0/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q0/acquire_line
              ports: [QACHANNELS/0/INPUT] 
            - iq_signal: q1/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q1/acquire_line
              ports: [QACHANNELS/0/INPUT] 
            - iq_signal: q2/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q2/acquire_line
              ports: [QACHANNELS/0/INPUT] 
            - iq_signal: q3/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q3/acquire_line
              ports: [QACHANNELS/0/INPUT]

        device_shfsg:
            - iq_signal: q0/drive_line
              ports: SGCHANNELS/0/OUTPUT
            - iq_signal: q0/drive_line_ef
              ports: SGCHANNELS/0/OUTPUT
            - iq_signal: q1/drive_line
              ports: SGCHANNELS/1/OUTPUT
            - iq_signal: q1/drive_line_ef
              ports: SGCHANNELS/1/OUTPUT
            - iq_signal: q2/drive_line
              ports: SGCHANNELS/2/OUTPUT
            - iq_signal: q2/drive_line_ef
              ports: SGCHANNELS/2/OUTPUT
            - iq_signal: q3/drive_line
              ports: SGCHANNELS/3/OUTPUT
            - iq_signal: q3/drive_line_ef
              ports: SGCHANNELS/3/OUTPUT
            - iq_signal: q4/drive_line
              ports: SGCHANNELS/4/OUTPUT
            - iq_signal: q4/drive_line_ef
              ports: SGCHANNELS/4/OUTPUT
            - iq_signal: q5/drive_line
              ports: SGCHANNELS/5/OUTPUT
            - iq_signal: q5/drive_line_ef
              ports: SGCHANNELS/5/OUTPUT
            - iq_signal: q6/drive_line
              ports: SGCHANNELS/6/OUTPUT
            - iq_signal: q6/drive_line_ef
              ports: SGCHANNELS/6/OUTPUT
            - iq_signal: q7/drive_line
              ports: SGCHANNELS/7/OUTPUT
            - iq_signal: q7/drive_line_ef
              ports: SGCHANNELS/7/OUTPUT
        
        device_pqsc:
            - to: device_shfqa
              port: ZSYNCS/2
            - to: device_shfsg
              port: ZSYNCS/1
            - internal_clock_signal
              
    """
    device_setup = DeviceSetup.from_descriptor(
        yaml_text=descriptor,
        server_host="localhost",
        server_port=8004,
        setup_name="jw_setup",
    )
    
    return device_setup

In [80]:
device_setup = make_device_setup()

In [94]:
device_setup.instruments[2]

PQSC(uid='device_pqsc', interface='1GbE', connections=[Connection(direction=IODirection.OUT, local_path=None, local_port='ZSYNCS/2', remote_path='device_shfqa', remote_port='0', signal_type=IOSignalType.ZSYNC), Connection(direction=IODirection.OUT, local_path=None, local_port='ZSYNCS/1', remote_path='device_shfsg', remote_port='0', signal_type=IOSignalType.ZSYNC)], server_uid='zi_server', address='DEV10084', reference_clock_source=<ReferenceClockSource.INTERNAL: 'internal'>, reference_clock=10000000.0)

In [82]:
device_setup.logical_signal_groups

{'q0': LogicalSignalGroup(uid='q0', logical_signals={'measure_line': LogicalSignal(uid='q0/measure_line', direction=IODirection.OUT, name='measure_line', calibration=None, path='/logical_signal_groups/q0/measure_line', physical_channel=PhysicalChannel(uid=device_shfqa/qachannels_0_output, type=PhysicalChannelType.IQ_CHANNEL, name=qachannels_0_output, path=/physical_channel_groups/device_shfqa/qachannels_0_output, calibration=None)), 'acquire_line': LogicalSignal(uid='q0/acquire_line', direction=IODirection.IN, name='acquire_line', calibration=None, path='/logical_signal_groups/q0/acquire_line', physical_channel=PhysicalChannel(uid=device_shfqa/qachannels_0_input, type=PhysicalChannelType.IQ_CHANNEL, name=qachannels_0_input, path=/physical_channel_groups/device_shfqa/qachannels_0_input, calibration=None)), 'drive_line': LogicalSignal(uid='q0/drive_line', direction=IODirection.OUT, name='drive_line', calibration=None, path='/logical_signal_groups/q0/drive_line', physical_channel=Physical

In [83]:
def define_calibration(config):
    
     # the calibration object will later be applied to the device setup
    my_calibration = Calibration()


    my_calibration["/logical_signal_groups/"+ name +"/drive_line"] = \
        SignalCalibration(
           # each logical signal can have an oscillator associated with it
            oscillator=Oscillator(
                frequency=config[name]['freq'],
                modulation_type=ModulationType.HARDWARE
            ),
            local_oscillator=Oscillator(
                frequency=config["lo"]['qb_lo'],
            ),
            range=config[name]["range"]-20
        )
    
    # readout drive line
    my_calibration["/logical_signal_groups/"+name+"/measure_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=config[name]['ro'],
                modulation_type=ModulationType.SOFTWARE
            ),
            port_delay=config["rr"]['ro_delay'],
            local_oscillator=Oscillator(
                frequency=config["lo"]['ro_lo'],
            ),
           range=config["rr"]["range"]
       )
    # acquisition line
    my_calibration["/logical_signal_groups/"+name+"/acquire_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=config[name]["ro"],
                modulation_type=ModulationType.SOFTWARE
            ),
            # add an offset between the readout pulse and the start of the data acquisition - to compensate for round-trip time of readout pulse 
            port_delay=config["rr"]['ro_delay'] + config["rr"]['ro_int_delay'],
            local_oscillator=Oscillator(
                frequency=config["lo"]['ro_lo'],
            ),
            range=config["rr"]["range"]
        )
  
    return my_calibration

In [84]:
config_calibration = define_calibration(config=config)
device_setup.set_calibration(config_calibration)

In [85]:
emulate=False
my_session = Session(device_setup=device_setup)
my_session.connect(do_emulation=emulate)

2023.03.21 16:07:57.531 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is d:\4. JWKim\laboneq\examples\laboneq_output\log
2023.03.21 16:07:57.532 laboneq.controller.controller  INFO   VERSION: laboneq 2.1.0
2023.03.21 16:07:57.533 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2023.03.21 16:07:57.748 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2023.03.21 16:07:58.941 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2023.03.21 16:07:59.131 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [86]:
amp_min = 0
amp_max = 1
amp_num = 61
n_average = 9

gaussian_pulse = pulse_library.gaussian(
    uid="gaussian_pulse", length=config[name]['drive_len'], amplitude=1)
drive_pulse = pulse_library.const(
    uid = "drive_pulse", length=config[name]['drive_len'], amplitude=1)
readout_pulse = pulse_library.const(
    length=config["rr"]["ro_len"], amplitude=config["rr"]['ro_amp'])
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=config["rr"]["ro_len"], amplitude=1.0)
def amplitude_rabi(rabi_sweep):
    exp_rabi = Experiment(
        uid="Amplitude Rabi",
        signals=[
            ExperimentSignal("drive"),
            ExperimentSignal("measure"),
            ExperimentSignal("acquire"),
        ],
    )

    with exp_rabi.acquire_loop_rt(uid="rabi_shots",
            count=pow(2, n_average),
            averaging_mode=AveragingMode.CYCLIC,
            acquisition_type=AcquisitionType.INTEGRATION):
        with exp_rabi.sweep(uid="rabi_sweep", parameter=LinearSweepParameter(uid="rabi_amp", start=amp_min, stop=amp_max, count=amp_num)):
            with exp_rabi.section(uid="qubit_excitation",
                                    alignment=SectionAlignment.RIGHT):
                 exp_rabi.play(signal="drive", pulse=gaussian_pulse, amplitude=rabi_sweep)
            with exp_rabi.section(uid="qubit_readout"):
                 exp_rabi.play(signal="measure", pulse=readout_pulse, phase=config[name]["ro_phase"])
                 exp_rabi.acquire(
                        signal="acquire",
                        handle="q0_rabi",
                        kernel=readout_weighting_function,
                    )
                 exp_rabi.reserve("drive")
            with exp_rabi.section(uid="delay", length=config["rr"]['relax']):
                 exp_rabi.reserve(signal="measure")
                
    return exp_rabi

In [87]:
exp_calibration = Calibration()
exp_calibration["drive"] = SignalCalibration(
        oscillator = Oscillator(
                frequency=config[name]["freq"],
                modulation_type=ModulationType.HARDWARE,
                ),
            )

exp_calibration["measure"] = SignalCalibration(
        oscillator = Oscillator(
                frequency=config[name]["ro"],
                modulation_type=ModulationType.SOFTWARE,
                ),
            )
exp_calibration["acquire"] = SignalCalibration(
        oscillator = Oscillator(
                frequency=config[name]["ro"],
                modulation_type=ModulationType.SOFTWARE,
                ),
            )
signal_map = {"drive": "/logical_signal_groups/"+name+"/drive_line",
        "measure": "/logical_signal_groups/"+name+"/measure_line",
        "acquire": "/logical_signal_groups/"+name+"/acquire_line",
        }

In [88]:
rabi_sweep=LinearSweepParameter(uid="rabi_amp", start=amp_min, stop=amp_max, count=amp_num)
exp_rabi = amplitude_rabi(rabi_sweep)
exp_rabi.set_calibration(exp_calibration)
exp_rabi.set_signal_map(signal_map)

compiled_experiment = my_session.compile(exp_rabi)
my_results = my_session.run(compiled_experiment)

2023.03.21 16:08:51.802 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.03.21 16:08:51.913 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 706
2023.03.21 16:08:51.914 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 56800
2023.03.21 16:08:51.914 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.03.21 16:09:03.557 laboneq.controller.util        CRITICAL PQSC:DEV10084: Node '/dev10084/zsyncs/2/connection/status' didn't switch to '2' within 10s. Last value: 0


2023.03.21 16:09:03.557 laboneq.controller.util        CRITICAL PQSC:DEV10084: Node '/dev10084/zsyncs/2/connection/status' didn't switch to '2' within 10s. Last value: 0


LabOneQControllerException: PQSC:DEV10084: Node '/dev10084/zsyncs/2/connection/status' didn't switch to '2' within 10s. Last value: 0